# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [8]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: request returned Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/_ping, check if the server supports the requested API version.
See 'docker run --help'.


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [2]:
!curl localhost

#the curl command should fail on our system 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 80 after 2538 ms: Could not connect to server


Lets run it on the docker container 

In [ ]:
!docker exec -it nginx /bin/sh -c "curl google.com"



* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [3]:
!docker network


Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [4]:
!docker network ls 

request returned Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/v1.47/networks, check if the server supports the requested API version


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [5]:
!docker network inspect none 

[]


request returned Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/v1.47/networks/none, check if the server supports the requested API version


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [ ]:
#!pip install docker

In [ ]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Lets create a quick code to list the containers using the docker client

In [9]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

adminer - running
mysql - running


Here you can see the running container and no surprise it is the nginx we started before


In [16]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


NotFound: 404 Client Error for http+docker://localnpipe/v1.47/containers/nginx/json: Not Found ("No such container: nginx")

again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [6]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

docker: request returned Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/_ping, check if the server supports the requested API version.
See 'docker run --help'.


## The following needs to be run in the terminal

In [7]:
!docker run -d --name adminer -p 8080:8080 adminer


docker: request returned Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/_ping, check if the server supports the requested API version.
See 'docker run --help'.


Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [ ]:
#!docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


In [ ]:
#!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [ ]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


## Setting up the network

In [ ]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

In [ ]:
!docker network inspect yourNetwork

In [ ]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


In [3]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

docker: Error response from daemon: Conflict. The container name "/adminer" is already in use by container "d96170b4bdc309304da5e504bdb3dffc81f90e944abc9dd96f44b0e12a9ea11f". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [ ]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

In [ ]:
#!docker inspect mysql |grep -C2 " \"Networks\""
#!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

In [ ]:
#!docker inspect adminer| grep -A2 " \"Networks\""
#!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
#!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 



In [1]:
import docker

client = docker.from_env()
print("Docker version:", client.version()['Version'])

# given example code
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")



def check_all_containers_IP():
    containers = client.containers.list(all=True)# list all containers

    for container in containers:
        show_container_ip(container_name= container.name)

check_all_containers_IP()




c:\Users\mrakl\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\mrakl\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\mrakl\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


DockerException: Error while fetching server API version: 500 Server Error for http+docker://localnpipe/version: Internal Server Error


## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/



In [ ]:
import docker
import time

client = docker.from_env()
print("Docker version:", client.version()['Version'])

# given example code
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")



def check_all_containers_IP():
    containers = client.containers.list(all=True)# list all containers

    for container in containers:
        show_container_ip(container_name= container.name)

check_all_containers_IP()



def adminer_keep_running():
    
    try:
        while(True):
            if container.status != "running":
                client.containers.run(container)
                print(f"restarted {container}")
            else:
                print(f"{container} is running")
            time.sleep(.1)

    except KeyboardInterrupt:
        print("keyboard interrupt")
    except Exception as e:
        print(f"error: {Exception}")


adminer_keep_running()

Docker version: 27.5.1
adminer in network 'yourNetwork' has IP: 172.18.0.3
mysql in network 'yourNetwork' has IP: 172.18.0.2
error: <class 'Exception'>


In [ ]:
containers = client.containers.list(all=True)# list all containers
for container in containers:
    print(container)

def find_name_for_process(container_name):
    containers = client.containers.list(all=True)
    for container in containers:
        if container_name == container.name:


<Container: d96170b4bdc3>
<Container: 1b1d3e07e50f>
